In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# AddSubtractComp

[AddSubtractComp](../../../_srcdocs/packages/components/add_subtract_comp) performs elementwise addition or subtraction between two or more compatible inputs.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    result = a * \textrm{scaling factor}_a + b * \textrm{scaling factor}_b + c * \textrm{scaling factor}_c + ...
$$

## AddSubtractComp Constructor

The call signature for the `AddSubtractComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.add_subtract_comp.AddSubtractComp.__init__
        :noindex:
```

## Using the AddSubtractComp

The `add_equation` method is used to set up a system of inputs to be added/subtracted (with scaling factors).
Each time the user adds an equation, all of the inputs and outputs must be of identical shape (this is a requirement for element-wise addition/subtraction).
The units must also be compatible between all inputs and the output of each equation.

## Method Signature

```{eval-rst}
    .. automethod:: openmdao.components.add_subtract_comp.AddSubtractComp.add_equation
        :noindex:
```

## AddSubtractComp Example

In the following example AddSubtractComp is used to add thrust, drag, lift, and weight forces. Note the scaling factor of -1 for the drag force and weight force.

In [ ]:
import numpy as np
import openmdao.api as om

n = 3

p = om.Problem()
model = p.model

# Construct an adder/subtracter here. create a relationship through the add_equation method
adder = om.AddSubtractComp()
adder.add_equation('total_force', input_names=['thrust', 'drag', 'lift', 'weight'],
                   vec_size=n, length=2, scaling_factors=[1, -1, 1, -1], units='kN')
# Note the scaling factors. we assume all forces are positive sign upstream

# The vector represents forces at 3 time points (rows) in 2 dimensional plane (cols)
p.model.add_subsystem(name='totalforcecomp', subsys=adder,
                      promotes_inputs=['thrust', 'drag', 'lift', 'weight'])

p.setup()

# Set thrust to exceed drag, weight to equal lift for this scenario
p['thrust'][:, 0] = [500, 600, 700]
p['drag'][:, 0] = [400, 400, 400]
p['weight'][:, 1] = [1000, 1001, 1002]
p['lift'][:, 1] = [1000, 1000, 1000]

p.run_model()

# Verify the results
expected_i = np.array([[100, 200, 300], [0, -1, -2]]).T
print(p.get_val('totalforcecomp.total_force', units='kN'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('totalforcecomp.total_force', units='kN'), expected_i)